# Date and time

Representing **time** is very important in science but also a challenging objective to achieve consistently.

Need to define:
- range: what is the longest timespan we have to cover? Maybe the age of the universe... ~14e9 yr.
- precision/accuracy: what is the shortest timescale we need to represent? Physics experiments often go down to sub-nanosecond accuracy, maybe picoseconds!

Remember when we talked about floating point numbers: big numbers and high resolution do not go well together!

## Timing matters
There are different "sources" for what the current time is:
- **Universal Coordinated Time** (UTC): is the standard, universal, official time reference. It is not strictly linear: leap seconds are added to compensate for the slowing in the Earth rotation. The day where the leap second is added, the time goes up to `23:59:60`).
- **Temps Atomique International** (TAI) is the actual instrumental time, as measured by atomic clocks. It has no notion of leap seconds so it is always ahead. The `TAI - UTC` difference gives the current number of leap seconds (37). At some point in the far-far future, TAI will be midnight and UTC will be noon (or maybe not, it seems metrologists are abandoning the leap seconds).
- **GPS - Global Positioning System** has its zero on the UTC time at 1980-01-06, so it has a different and fixed number of leap seconds compared to UTC.

Note: GPS is the most common way to get a reasonably accurate time reference at any surface location, requiring just some relatively inexpensive hardware. A computer usually uses NTP (Network Time Protocol) over the Internet, although the network latency can cause small inaccuracies (tens of ms level).

### Summary
- `TAI - UTC` = 37 leap seconds as of today.
- `TAI - GPS` = 19 leap seconds at 1980, January 6th.
- `GPS - UTC` = 18 leap seconds as of today.



### The UNIX way

- UNIX systems traditionally represent the UTC time in number of seconds since 01-01-1970 using one integer (32 bits).
- Where do leap seconds (e.g. 23:59:60) go? They are cannot be represented in a UNIX timestamp, this means any time a leap second occurs, two "physical" seconds will have the same timestamp. Physics experiment need to deal carefully with these situations!
- 32 bits would only cover the time up to 2038, so modern computers use 64 bits!

### How to represent decimal times?
We can use two values. Let's write a small `Timestamp` class.

In [ ]:
class Timestamp:
    def __init__(self, s : int, ns : int):
        self.s = s
        self.ns = ns

    def __str__(self):
        return f"{self.s}.{self.ns:8d}"

In [ ]:
TSTAMP = 1673608072 # A UNIX timestamp reasonably near to present time.

t = Timestamp(TSTAMP, 100000001) 
print(t)

With a bit of additional methods, this could be an excellent way to represent a very accurate time on a large time scale (especially if using 64 bit integers). In fact this is what the ROOT (C++) Data Analysis framework of CERN uses for their `TTimeStamp` class).

### The `python` way

In [ ]:
import datetime as _datetime 
# we shall alias it as _datetime because for all practical purposes we will do `import datetime from datetime` later!

In [ ]:
a = _datetime.date(year=2023, month=1, day=1)
b = _datetime.time(hour=14, minute=0, second=0, microsecond=int(1e5))
c = _datetime.time(hour=14, minute=0, second=0, microsecond=0)
print(a, b)

In [ ]:
# We cannot do arithmetics with `time` objects. After all, a time object does not have a date, hence the difference between two times is undefined.
# d = b-c

For all practical purposes, the `datetime` class combines date and time:

In [ ]:
from datetime import datetime
c = datetime(year=2023, month=1, day=1, hour=14, minute=0, second=0, microsecond=0)
d = datetime(year=2023, month=1, day=1, hour=14, minute=0, second=0, microsecond=int(1e5))

diff = d-c

print(type(diff), diff)

The `timedelta` class provides a way to represent time differences:

In [ ]:
from datetime import  timedelta

c = datetime(year=2023, month=1, day=1, hour=14, minute=0, second=0, microsecond=0)

times = []
for delta in range(10):
    times.append(c + timedelta(microseconds=delta * int(1e5)))

for time in times:
    print(time)

We can get the current time from the computer RTC (real time clock):

In [ ]:
t = datetime.utcnow()

print(t)

We can also create an object from the UNIX timestamp we have used before:

In [ ]:
u = datetime.utcfromtimestamp(TSTAMP)
print(u)

### The `numpy` way

The python `datetime` class is quite versatile, but is not really meant for scientific computing. In fact, it does not support arrays or sub-microsecond accuracy. `numpy` has reimplemented it with `datetime64`.

In [ ]:
import numpy as np

# Creating a datetime64 object. The passed arguments define a time referenced to the start of the UNIX timestamp (1/1/1970).

a = np.datetime64(53, 'Y') 

print(a)

A more convenient way to create such an item, is using an ISO 8601 string:

In [ ]:
b = np.datetime64('2023-01-13T14:00')
print(b)

We can also use decimal numbers up to nanoseconds:

In [ ]:
d = np.datetime64("1999-01-01T00:00:00.123456789")
print(d)

### Notes
- The `numpy.datetime64` can be used in numpy arrays and is therefore more efficient for scientific computing;
- There will be some cases in which you need to deal with a large number of time values spanning for months of years, in such cases it is a good practice to use a dedicated time format such as `numpy.datetime64`;
- However, if your experimental data only span a few hours or days, it may be more convenient to define your own custom "zero time" and use a floating point number (but be careful to check that the accuracy is more than sufficient) for the number of seconds.
- **Unfortunately, there is no standard.**: `python`, `numpy`, `astropy` and `pandas` have all their own time/timestamp format. For science, `astropy.time` is even more sophisticated than `numpy.datetime64`. Note also that `JSON` does not support writing (serialization) of timestamp out of the box, to save timestamps in a text format you need to define your own (or use an ISO-like as in "1999-01-01T00:00:00.123456789").